In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
FOLDERNAME = "Colab\ Notebooks/fetch-data"

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/Colab Notebooks/WeHelp/Final Project/Datasets


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from sklearn.model_selection import train_test_split

In [ ]:
class ResumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts  = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids":      enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels":         torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
# Hyperparameters
MODEL_NAME = "xlm-roberta-base"
DATA_PATH  = "cleaned/BERT/FAAMG_bert.csv"
BATCH_SIZE = 8
LR         = 3e-5
NUM_EPOCHS = 5
MAX_LEN    = 512
DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv(DATA_PATH)
df = df[["text","label"]].dropna()
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.1,
    stratify=df["label"],
    random_state=42
)

# Initialize tokenizer and set special tokens
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
special_tokens_dict = {
    "additional_special_tokens": ["[EDU]", "[EXP]", "[PRO]", "[COU]", "[PUB]", "[CER]"]
}
tokenizer.add_special_tokens(special_tokens_dict)

# Establish Dataset/Dataloader
train_ds = ResumeDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_ds   = ResumeDataset(val_texts,   val_labels,   tokenizer, MAX_LEN)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

# 4. Load model and resize token embeddings
num_labels = len(set(train_labels))
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)
model.resize_token_embeddings(len(tokenizer))
model.to(DEVICE)

# Set optimizer, scheduler, loss
optimizer = AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)
loss_fn = torch.nn.CrossEntropyLoss()

# Start training
for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids      = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels         = batch["labels"].to(DEVICE)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    correct  = 0
    total    = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids      = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels         = batch["labels"].to(DEVICE)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            logits = outputs.logits
            loss   = loss_fn(logits, labels)

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc      = correct / total

    print(
        f"Epoch {epoch}/{NUM_EPOCHS}  "
        f"Train Loss: {avg_train_loss:.4f}  "
        f"Val Loss:   {avg_val_loss:.4f}  "
        f"Val Acc:    {val_acc:.4f}"
    )

save_dir = "./BERT_models/bert_pytorch_finetuned"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"✅ Model and tokenizer saved to: {save_dir}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Epoch 1/5  Train Loss: 0.8949  Val Loss:   0.7618  Val Acc:    0.7083
Epoch 2/5  Train Loss: 0.7350  Val Loss:   0.7188  Val Acc:    0.7242
Epoch 3/5  Train Loss: 0.6773  Val Loss:   0.6990  Val Acc:    0.7360
Epoch 4/5  Train Loss: 0.6053  Val Loss:   0.7000  Val Acc:    0.7283
Epoch 5/5  Train Loss: 0.5220  Val Loss:   0.7357  Val Acc:    0.7274
✅ Model and tokenizer saved to: ./BERT_models/bert_pytorch_finetuned
